In [ ]:
# Instalação das bibliotecas
!pip install -q google-generativeai
!pip install -q pdfminer.six
!pip install -q fpdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Importações necessárias
import google.generativeai as genai
from pdfminer.high_level import extract_text
from google.colab import userdata
import json
from fpdf import FPDF

In [ ]:
# Configuração da API Key do Gemini de forma segura
try:
    # Armazenei a API Key no Colab Secrets com o nome "GEMINI_API_KEY"
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print(" API Key do Gemini configurada com sucesso!")
except userdata.SecretNotFoundError:
    print(' Secret "GEMINI_API_KEY" não encontrado. Adicione a chave ao painel de Secrets do Colab.')

 API Key do Gemini configurada com sucesso!


In [ ]:
# Instanciando o modelo Gemini

model = genai.GenerativeModel(model_name="gemini-2.0-flash")

In [ ]:
# Função para extrair texto do PDF (sem alteração)
caminho_pdf = "/content/Cenário Econômico.pdf"
def extrair_texto_de_pdf(caminho_pdf):
    texto_extraido = extract_text(caminho_pdf)
    texto_limpo = ' '.join(texto_extraido.split())
    return texto_limpo

In [ ]:
# Célula 6: Função genérica para chamar a API do Gemini
def obter_resposta_gemini(prompt, persona=None):
    """
    Função para enviar um prompt para o modelo Gemini e retornar a resposta.
    """
    # Combina a persona com o prompt, se fornecida.
    if persona:
        prompt_completo = f"{persona}\n\n{prompt}"
    else:
        prompt_completo = prompt

    try:
        resposta = model.generate_content(prompt_completo)
        return resposta.text
    except Exception as e:
        print(f"Ocorreu um erro ao chamar a API do Gemini: {e}")
        return None

In [ ]:
# Célula 7: Extraindo o texto do arquivo
texto_limpo = extrair_texto_de_pdf(caminho_pdf)
print("📄 Texto extraído do PDF e normalizado.")

📄 Texto extraído do PDF e normalizado.


In [ ]:
texto_limpo

'Cenário Global em 2024 O início de 2023 mostrou-se extremamente desafiador, com grandes incertezas sobre o cenário macroeconômico global. Diante da expressiva elevação da inflação, acreditava-se que uma forte desaceleração da atividade seria necessária para desinflacionar as economias, o que demandou um forte ciclo de aperto monetário sincronizado no mundo. Em contrapartida, os riscos financeiros dessa forte alta de juros, como os relacionados à solvência em um contexto de endividamento elevado, geraram preocupações, especialmente após a quebra do Silicon Valley Bank no primeiro semestre do ano. O receio de uma recessão se instalou nos mercados. Mas, contrariando as previsões, terminamos o ano de 2023 com uma atividade resiliente, ainda que em desaceleração, e uma dinâmica desinflacionária muito mais benigna que as expectativas de mercado. Mais surpreendente que a significativa redução da inflação global foi a propagação da desinflação dos bens comercializáveis para os núcleos de infl

In [ ]:
# Célula 8: Prompt para extração de entidades
prompt_entidades = f"""
Sua tarefa é extrair todas as economias (países ou blocos econômicos como 'Europa') citadas no texto abaixo.

Texto:
'''{texto_limpo}'''

Observação: se uma economia for citada mais de uma vez, inclua-a apenas uma vez.
Formate sua resposta como uma lista JSON, como no exemplo: ["economia_1", "economia_2", "economia_3"]
Responda apenas com a lista JSON, sem nenhum texto ou formatação adicional.
"""

In [ ]:
# Célula 9: Obtendo a resposta e processando
print("🔎 Extraindo entidades com Gemini...")
resposta_entidades_str = obter_resposta_gemini(prompt_entidades)

🔎 Extraindo entidades com Gemini...


In [ ]:
resposta_entidades_str

'["Estados Unidos", "Brasil", "Europa", "China"]'

In [ ]:
# Célula 10: Convertendo a string JSON em uma lista Python de forma segura
try:
    lista_economias = json.loads(resposta_entidades_str)
    print(f"Economias citadas no texto: {lista_economias}")
except (json.JSONDecodeError, TypeError):
    print(" Falha ao decodificar a resposta. Tentando extrair a lista manualmente.")
    # Fallback caso a resposta não seja um JSON perfeito, embora raro com os prompts corretos
    lista_economias = eval(resposta_entidades_str)
    print(f"Economias citadas no texto (via eval): {lista_economias}")


Economias citadas no texto: ['Estados Unidos', 'Brasil', 'Europa', 'China']


In [ ]:
# Célula 11: Preparando a lista para o próximo prompt
resposta_em_topicos = '* ' + '\n* '.join(lista_economias)

In [ ]:
print(resposta_em_topicos)

* Estados Unidos
* Brasil
* Europa
* China


In [ ]:
# Célula 12: Prompt combinado para análise de sentimentos e resumo
prompt_sentimento_e_resumo = f"""
Sua tarefa é analisar o sentimento e criar um resumo para cada uma das economias listadas, com base no texto fornecido.

Economias para Análise:
{resposta_em_topicos}

Texto de Referência:
'''{texto_limpo}'''

Instruções:
1.  Para cada economia, determine o sentimento geral (positivo, negativo ou neutro) com base no contexto.
2.  Para cada economia, escreva um resumo conciso de um parágrafo.
3.  Formate sua resposta como um único objeto JSON. As chaves devem ser os nomes das economias. O valor para cada chave deve ser outro objeto contendo "sentimento" e "resumo".

Exemplo de formato da resposta:
{{
  "economia_1": {{
    "sentimento": "positivo",
    "resumo": "O resumo da economia 1 vai aqui."
  }},
  "economia_2": {{
    "sentimento": "negativo",
    "resumo": "O resumo da economia 2 vai aqui."
  }}
}}

Responda apenas com o objeto JSON.
"""

In [ ]:
prompt_sentimento_e_resumo

'\nSua tarefa é analisar o sentimento e criar um resumo para cada uma das economias listadas, com base no texto fornecido.\n\nEconomias para Análise:\n* Estados Unidos\n* Brasil\n* Europa\n* China\n\nTexto de Referência:\n\'\'\'Cenário Global em 2024 O início de 2023 mostrou-se extremamente desafiador, com grandes incertezas sobre o cenário macroeconômico global. Diante da expressiva elevação da inflação, acreditava-se que uma forte desaceleração da atividade seria necessária para desinflacionar as economias, o que demandou um forte ciclo de aperto monetário sincronizado no mundo. Em contrapartida, os riscos financeiros dessa forte alta de juros, como os relacionados à solvência em um contexto de endividamento elevado, geraram preocupações, especialmente após a quebra do Silicon Valley Bank no primeiro semestre do ano. O receio de uma recessão se instalou nos mercados. Mas, contrariando as previsões, terminamos o ano de 2023 com uma atividade resiliente, ainda que em desaceleração, e u

In [ ]:
# Célula 13: Definindo a persona e obtendo a resposta
print(" Analisando sentimentos e criando resumos...")
persona_economista = "Você é um economista sênior com vasta experiência em análise macroeconômica global."
resposta_completa_str = obter_resposta_gemini(prompt_sentimento_e_resumo, persona=persona_economista)

📊 Analisando sentimentos e criando resumos...


In [ ]:
resposta_completa_str

'```json\n{\n  "Estados Unidos": {\n    "sentimento": "positivo",\n    "resumo": "A economia dos EUA apresenta um cenário de \'pouso suave\' com rebalanceamento gradual no mercado de trabalho e uma atividade ligeiramente mais fraca, mas ainda em crescimento. A inflação em declínio levou o FOMC a sinalizar possíveis cortes nas taxas de juros em 2024, com o mercado precificando reduções significativas. O ambiente geral é de otimismo cauteloso, impulsionado pela desinflação e pela expectativa de uma política monetária mais flexível."\n  },\n  "Brasil": {\n    "sentimento": "positivo",\n    "resumo": "O Brasil demonstra resiliência econômica, com um crescimento do PIB em torno de 3% pelo segundo ano consecutivo e uma inflação mais baixa que o esperado. Esse cenário favorece o ciclo de flexibilização da política monetária em curso, com expectativas de cortes adicionais na taxa Selic. A perspectiva é positiva, sustentada pela continuidade dos cortes na Selic, resiliência do mercado de trabal

In [ ]:
# Célula 14: Processando a resposta JSON

if resposta_completa_str:
    try:
        # Encontra o início do JSON (o primeiro '{') e o fim (o último '}')
        inicio_json = resposta_completa_str.find('{')
        fim_json = resposta_completa_str.rfind('}') + 1

        # Extrai apenas a parte que corresponde ao JSON
        json_str_limpo = resposta_completa_str[inicio_json:fim_json]


        # Processa a string JSON limpa
        analise_completa = json.loads(json_str_limpo)

        print(" Análise e resumo concluídos com sucesso!")

        # Exemplo de como acessar os dados (sem alteração aqui)
        print("\nAnálise para o Brasil:")
        print(f"  Sentimento: {analise_completa['Brasil']['sentimento']}")
        print(f"  Resumo: {analise_completa['Brasil']['resumo']}")

    except (json.JSONDecodeError, TypeError, KeyError) as e:
        print(" Falha ao processar a resposta JSON do Gemini.")
        print(f"Tipo de erro: {type(e).__name__}")
        print("\nResposta recebida (bruta):")
        print(resposta_completa_str)
else:
    print(" Nenhuma resposta foi recebida da API.")

 Análise e resumo concluídos com sucesso!

Análise para o Brasil:
  Sentimento: positivo
  Resumo: O Brasil demonstra resiliência econômica, com um crescimento do PIB em torno de 3% pelo segundo ano consecutivo e uma inflação mais baixa que o esperado. Esse cenário favorece o ciclo de flexibilização da política monetária em curso, com expectativas de cortes adicionais na taxa Selic. A perspectiva é positiva, sustentada pela continuidade dos cortes na Selic, resiliência do mercado de trabalho e desinflação, embora existam riscos relacionados à desaceleração chinesa e fatores climáticos.


In [ ]:
# Célula 15: Reescrevendo para público técnico/especializado
print("\n Adaptando resumos para público técnico...")
resumo_tecnico = {}
persona_editor_tecnico = "Você é um editor experiente de uma revista técnica focada em economia. Reescreva o texto a seguir mantendo a precisão técnica e um tom formal e enxuta."

for economia, dados in analise_completa.items():
    resumo_original = dados['resumo']
    prompt_comunicacao = f"Reescreva o seguinte texto para um público de especialistas em economia:\n\n'''{resumo_original}'''"
    resumo_reescrito = obter_resposta_gemini(prompt_comunicacao, persona=persona_editor_tecnico)
    resumo_tecnico[economia] = resumo_reescrito

print("Resumo técnico para a Europa:", resumo_tecnico.get("Europa"))


 Adaptando resumos para público técnico...
Resumo técnico para a Europa: "A economia europeia exibe um quadro de desinflação concomitante à moderação da atividade econômica. A rápida transmissão da política monetária implementada justifica o crescimento aquém do potencial observado em 2023 e a subestimação da trajetória inflacionária. Antecipa-se o início do ciclo de flexibilização monetária em 2024, com projeções indicando a convergência da inflação à meta estabelecida em 2025, sinalizando uma estabilização gradual do cenário macroeconômico."



In [ ]:
# Célula 16: Reescrevendo para público geral/leigo
print("\n Adaptando resumos para público geral...")
resumo_leigo = {}
persona_editor_geral = "Você é um editor de uma revista de notícias para o público geral. Reescreva o texto a seguir em uma linguagem simples, enxuta e acessível, explicando termos complexos se necessário."

for economia, dados in analise_completa.items():
    resumo_original = dados['resumo']
    prompt_comunicacao = f"Reescreva o seguinte texto para um público sem conhecimento de economia:\n\n'''{resumo_original}'''"
    resumo_reescrito = obter_resposta_gemini(prompt_comunicacao, persona=persona_editor_geral)
    resumo_leigo[economia] = resumo_reescrito

print("Resumo para leigos sobre a Europa:", resumo_leigo.get("Europa"))


✍️ Adaptando resumos para público geral...
Resumo para leigos sobre a Europa: **A Europa em Apuros: Inflação caindo, mas economia fraca**

A Europa está passando por um momento delicado. Os preços dos produtos e serviços (a chamada **inflação**) estão finalmente começando a cair, o que é uma boa notícia. No entanto, a economia não está indo bem.

Para entender o que está acontecendo, pense assim: o **Banco Central Europeu** (o banco dos bancos na Europa) tem usado uma ferramenta chamada **política monetária** para controlar a inflação. Essa ferramenta, como aumentar ou diminuir as taxas de juros, tem funcionado mais rápido do que o esperado.

O resultado é que a economia europeia cresceu pouco em 2023, e a inflação caiu mais do que se imaginava. Isso mostra que as medidas tomadas pelo Banco Central tiveram um impacto forte e rápido.

O que esperar para o futuro? A previsão é que o Banco Central Europeu comece a **aliviar as restrições financeiras** em 2024. Isso significa, por exemplo

In [ ]:
# Célula 17: Função para gerar o PDF (levemente ajustada)
def salvar_dicionario_em_pdf(dicionario_resumos, dicionario_original_analise, caminho_pdf, tipo_resumo):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", style='B', size=16)
    pdf.cell(0, 10, f"Resumo Econômico - {tipo_resumo}", ln=True, align='C')
    pdf.ln(10)

    for pais, resumo in dicionario_resumos.items():
        # Acessa o sentimento do dicionário original
        sentimento = dicionario_original_analise.get(pais, {}).get('sentimento', 'Neutro')

        pdf.set_font("Arial", style='BU', size=12)
        pdf.cell(0, 8, f"{pais}", ln=True)

        pdf.set_font("Arial", style='I', size=10)
        pdf.cell(0, 6, f"Sentimento: {sentimento.title()}", ln=True)

        pdf.set_font("Arial", size=10)
        # Usando multi_cell para texto que pode quebrar linha e corrigindo a codificação
        pdf.multi_cell(0, 6, resumo)
        pdf.ln()

    pdf.output(caminho_pdf)
    print(f"📄 PDF '{caminho_pdf}' gerado com sucesso.")

# Gerando os dois relatórios
salvar_dicionario_em_pdf(resumo_tecnico, analise_completa, "resumos_paises_tecnico.pdf", "Versão Técnica")
salvar_dicionario_em_pdf(resumo_leigo, analise_completa, "resumos_paises_leigo.pdf", "Versão para o Público Geral")

📄 PDF 'resumos_paises_tecnico.pdf' gerado com sucesso.
📄 PDF 'resumos_paises_leigo.pdf' gerado com sucesso.
